# Bike Sharing: Predicción de Demanda Horaria

In [ ]:
explicacion

## A)

In [ ]:
import pandas as pd
import numpy as np
dftrain = pd.read_csv(’bike_sharing_train.csv’)
dfval = pd.read_csv(’bike_sharing_val.csv’)
dftest = pd.read_csv(’bike_sharing_test.csv’)
ntrain = len(dftrain)
nval = len(dftrain) + len(dfval)
df = pd.concat([dftrain,dfval,dftest])
print(’\nSummary - dataframe completo:\n’)
print(df.describe())
df[’hour’] = pd.to_datetime(df[’datetime’]).apply(lambda x: x.strftime(’%H’))
df[’hour’] = pd.to_numeric(df[’hour’])

## B)

In [ ]:
from sklearn.tree import DecisionTreeRegressor as Tree
import matplotlib.pyplot as plt

def eval_bikemodel(y_predict,y_true):
    diff = np.log(y_predict+1.0) - np.log(y_true+1.0)
    return np.sqrt(np.sum(np.square(diff))/len(y_predict))

Xdf=df.ix[:,[’season’,’holiday’,’workingday’,’weather’,’temp’,’atemp’,
’humidity’,’windspeed’,’hour’]]
Ydf=df.ix[:,’count’]
X_train = Xdf[0:ntrain].values
X_val = Xdf[ntrain:nval].values
X_test = Xdf[nval:].values
Y_train = Ydf[0:ntrain].values
Y_val = Ydf[ntrain:nval].values
Y_test = Ydf[nval:].values
model = Tree(random_state=0)
model.fit(X_train,Y_train)
score_test = model.score(X_test,Y_test)
print "SCORE TEST=%f"%score_test
Y_pred_train = model.predict(X_train)
Y_pred_val = model.predict(X_val)
Y_pred_test = model.predict(X_test)
kagg_train = eval_bikemodel(Y_pred_train,Y_train)
kagg_val = eval_bikemodel(Y_pred_val,Y_val)
kagg_test = eval_bikemodel(Y_pred_test,Y_test)
print "KAGG EVAL TRAIN =%f"%kagg_train
print "KAGG EVAL TEST =%f"%kagg_test

## C)

In [ ]:
model = Tree(random_state=0,max_depth=20)
Y_pred_val = model.predict(X_val)
kagg_val = eval_bikemodel(Y_pred_val,Y_val)
print "KAGG EVAL VAL =%f"%kagg_val

## D)

In [ ]:
df[’cday’] = pd.to_datetime(df[’datetime’]).dt.dayofweek#0:lunes,6:domingo
df[’cday’] = pd.to_numeric(df[’cday’])
Xdf=df.ix[:,[’season’,’holiday’,’workingday’,’weather’,’temp’,’atemp’,
’humidity’,’windspeed’,’hour’,’cday’]]

## E)

In [ ]:
#load dataframes as before ...
df = pd.concat([dftrain,dfval,dftest])
df[’hour’] = pd.to_datetime(df[’datetime’]).apply(lambda x: x.strftime(’%H’))
df[’cday’] = pd.to_datetime(df[’datetime’]).dt.dayofweek
df[’hour’] = pd.to_numeric(df[’hour’])
df[’cday’] = pd.to_numeric(df[’cday’])
Xdf=df.ix[:,[’season’,’holiday’,’workingday’,’weather’,’temp’,’atemp’,
’humidity’,’windspeed’,’hour’,’cday’]]
#PASO IMPORTANTE MAS ABAJO ...
Xdf = pd.get_dummies(Xdf,columns=[’season’, ’weather’,’hour’,’cday’])
Ydf=df.ix[:,’count’]

from sklearn.preprocessing import StandardScaler
scalerX = StandardScaler()
X_train = scalerX.fit_transform(X_train)
X_val = scalerX.fit_transform(X_val)
X_test = scalerX.transform(X_test)

from sklearn.svm import SVR
model = SVR()
model.fit(X_train,Y_train)
Y_pred_train = model.predict(X_train)
Y_pred_val = model.predict(X_val)
Y_pred_test = model.predict(X_test)

## F)

In [ ]:
model = SVR(C=1,epsilon=0.01)
kagg_train = eval_bikemodel(Y_pred_train,Y_train)
kagg_val = eval_bikemodel(Y_pred_val,Y_val)
print "KAGG EVAL TRAIN =%f"%kagg_train
print "KAGG EVAL VAL =%f"%kagg_val

## G)

In [ ]:
from sklearn import cross_validation
k_fold = cross_validation.KFold(len(X_train),10)
score_cv = 0
for k, (train, val) in enumerate(k_fold):
model = #define your model, e.g. model = SVR(C=candidate)
model.fit(X_train[train], Y_train[train])
Ypred_val = model.predict(X_train[val])
Ytrue_val = Y_train[val]
score_fold = eval_bikemodel(Ypred_val,Ytrue_val)
score_cv += score_fold
score_cv = score_cv / 10

## H)

In [ ]:
Ydf=df.ix[:,’count’] #demanda total
Ydf=df.ix[:,’registered’] #demanda registrada
Ydf=df.ix[:,’casual’] #demanda casual

## I)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=10,max_depth=max_depth,random_state=0)